<a href="https://colab.research.google.com/github/abzokhattab/SpaceEfficientFolding/blob/main/SpaceEfficientAlg_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Requirements and Packaide Setup

In [ ]:
# Update package lists quietly
!sudo apt-get update -qq

# Install CMake (version 3.13+)
!sudo apt-get install -y cmake

# Install Boost libraries (including Boost.Python)
!sudo apt-get install -y libboost-all-dev

# Install CGAL and its dependencies
!sudo apt-get install -y libcgal-dev

# Install other essential build tools
!sudo apt-get install -y build-essential git

# Install Python development headers (required for building Python bindings)
!sudo apt-get install -y python3-dev

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libboost-all-dev is already the newest version (1.74.0.3ubuntu7).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libgmp-dev libgmpxx4ldbl libmpfr-dev
Suggested packages:
  libmpfi-dev libntl-dev gmp-doc libgmp10-doc libmpfr-doc
The following NEW packages will be installed:
  libcgal-dev libgmp-dev libgmpxx4ldbl libmpfr-

In [ ]:
# Install Python packages
!pip install shapely svgwrite parameterized svgelements libigl #igl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 48.3 MB/s eta 0:00:00


In [ ]:
# Clone the Packaide repository
!git clone https://github.com/DanielLiamAnderson/Packaide.git

Cloning into 'Packaide'...
remote: Enumerating objects: 583, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 583 (delta 51), reused 86 (delta 29), pack-reused 466 (from 1)
Receiving objects: 100% (583/583), 300.82 KiB | 2.69 MiB/s, done.
Resolving deltas: 100% (263/263), done.


In [ ]:
# Create a build directory and navigate into it
# always come back to the parent dir if you don't want a problem
%cd /content/Packaide
!mkdir build
%cd build

# Configure the build with CMake
!cmake -DBUILD_PYTHON_BINDINGS=ON ..

# Compile the library using all available CPU cores
!make -j$(nproc)
!make check
!sudo make install

%cd /content/Packaide/python
!pip install .

/content/Packaide
/content/Packaide/build
-- The CXX compiler identification is GNU 11.4.0
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- --------------- General configuration -------------
-- CMake Generator:                Unix Makefiles
-- Compiler:                       GNU 11.4.0
-- Build type:                     Release
-- CMAKE_CXX_FLAGS:                
-- CMAKE_CXX_FLAGS_DEBUG:          -g
-- CMAKE_CXX_FLAGS_RELEASE:        -O3 -DNDEBUG
-- CMAKE_CXX_FLAGS_RELWITHDEBINFO: -O2 -g -DNDEBUG -fno-omit-frame-pointer
-- CMAKE_EXE_LINKER_FLAGS          
-- CMAKE_INSTALL_PREFIX:           /usr/local
-- ---------------------------------------------------
CMake Warning at /usr/lib/x86_64-linux-gnu/cmake/CGAL/CGALConfig.cmake:92 (message):
  CGAL_DATA_DIR cannot be deduced, set the variable CGAL_DATA_DIR to set the
  default

#Test packaide Setup

In [ ]:
import packaide
print("Packaide imported successfully!")

Packaide imported successfully!


###We you could test if its working fine using this example usage from packaide readme documentation

In [ ]:
# Shapes are provided in SVG format
shapes = """
<svg viewBox="0 0 432.13 593.04">
  <rect width="100" height="50" />
  <rect width="50" height="100" />
  <ellipse rx="20" ry="20" />
</svg>
"""

# The target sheet / material is also represented as an SVG
# document. Shapes given on the sheet are interpreted as
# holes that must be avoided when placing new parts. In this
# case, a square in the upper-left-hand corner.
sheet = """
<svg width="300" height="300" viewBox="0 0 300 300">
  <rect x="0" y="0" width="100" height="100" />
</svg>
"""

# Attempts to pack as many of the parts as possible.
result, placed, fails = packaide.pack(
  [sheet],                  # A list of sheets (SVG documents)
  shapes,                   # An SVG document containing the parts
  tolerance = 2.5,          # Discretization tolerance
  offset = 5,               # The offset distance around each shape (dilation)
  partial_solution = True,  # Whether to return a partial solution
  rotations = 1,            # The number of rotations of parts to try
  persist = True            # Cache results to speed up next run
)

# If partial_solution was False, then either every part is placed or none
# are. Otherwise, as many as possible are placed. placed and fails denote
# the number of parts that could be and could not be placed respectively
print("{} parts were placed. {} parts could not fit on the sheets".format(placed, fails))

# The results are given by a list of pairs (i, out), where
# i is the index of the sheet on which shapes were packed, and
# out is an SVG representation of the parts that are to be
# placed on that sheet.
for i, out in result:
  with open('result_sheet_{}.svg'.format(i), 'w') as f_out:
    f_out.write(out)

3 parts were placed. 0 parts could not fit on the sheets


#Main

Below is a high level script that implements the pipeline. In this script we:

- Define several candidate cube nets as SVG strings (you can add more candidates as needed).
- Nest each candidate onto a blank sheet using Packaide.
Extract the geometry of the placed shapes to compute both the union area (the actual area of the unfolding) and the area of its minimal bounding rectangle (the envelope).
- Compute a utilization ratio (shape area divided by bounding box area) as a measure of how “tight” the unfolding is.
- Select and save the candidate with the highest utilization.

So far, we deploy:
- Geometry Utilities:
compute_union_polygon extracts and unions all placed polygons from the result.
compute_bounding_box_area computes the area of the minimal rectangle that encloses the union of the shapes.
compute_shape_area computes the total area of the shape.
compute_utilization calculates the ratio of the shape area to its bounding box area.

- Candidate Unfoldings:
Three simple cube nets are hardcoded as SVG strings. (could be extended with additional candidates.)

- Nesting via Packaide:
For each candidate, we call packaide.pack to nest the unfolding on a blank sheet. Packaide returns a result (an SVG with the placed shapes).

- Evaluation:
For each nesting result, the script computes the utilization ratio. The candidate with the highest ratio is considered the best (i.e., it minimizes wasted space as measured by the bounding box).

- Output:
The best candidate’s nesting result is printed to the console and saved to an SVG file named optimal_unfolding.svg.

> Below is a hard coded list of all unfolding candidates of a cube.

In [ ]:
from IPython.display import HTML, display

def display_svg(svg_str, width=200):
    """
    Display the SVG string in a container with the specified width.
    """
    html_str = f'<div style="width: {width}px;">{svg_str}</div>'
    display(HTML(html_str))

# Candidate 0: Cross net
candidate0 = """
<svg viewBox="0 0 200 200" xmlns="http://www.w3.org/2000/svg">
  <!-- Candidate 0: Cross net -->
  <rect x="50" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="0" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="100" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="0" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="100" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="150" width="50" height="50" stroke="black" fill="none"/>
</svg>
"""

# Candidate 1: Row-extended net
candidate1 = """
<svg viewBox="0 0 200 150" xmlns="http://www.w3.org/2000/svg">
  <!-- Candidate 1: Row-extended net -->
  <rect x="0" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="100" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="150" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="0" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="100" width="50" height="50" stroke="black" fill="none"/>
</svg>
"""

# Candidate 2: L-shaped net
candidate2 = """
<svg viewBox="0 0 150 150" xmlns="http://www.w3.org/2000/svg">
  <!-- Candidate 2: L-shaped net -->
  <rect x="50" y="0" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="0" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="100" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="0" y="100" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="100" width="50" height="50" stroke="black" fill="none"/>
</svg>
"""

# Candidate 3: Alternate L-shaped net
candidate3 = """
<svg viewBox="0 0 150 150" xmlns="http://www.w3.org/2000/svg">
  <!-- Candidate 3: Alternate L-shaped net -->
  <rect x="50" y="0" width="50" height="50" stroke="black" fill="none"/>
  <rect x="0" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="100" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="100" y="100" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="100" width="50" height="50" stroke="black" fill="none"/>
</svg>
"""

# Candidate 4: Offset net
candidate4 = """
<svg viewBox="0 0 150 200" xmlns="http://www.w3.org/2000/svg">
  <!-- Candidate 4: Offset net -->
  <rect x="0" y="0" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="0" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="100" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="100" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="150" width="50" height="50" stroke="black" fill="none"/>
</svg>
"""

# Candidate 5: Extended row with branch
candidate5 = """
<svg viewBox="0 0 200 150" xmlns="http://www.w3.org/2000/svg">
  <!-- Candidate 5: Extended row with branch -->
  <rect x="50" y="0" width="50" height="50" stroke="black" fill="none"/>
  <rect x="0" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="100" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="150" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="100" y="100" width="50" height="50" stroke="black" fill="none"/>
</svg>
"""

# Candidate 6: Vertical column attached off-center
candidate6 = """
<svg viewBox="0 0 150 200" xmlns="http://www.w3.org/2000/svg">
  <!-- Candidate 6: Vertical column attached off-center -->
  <rect x="50" y="0" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="0" y="100" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="100" width="50" height="50" stroke="black" fill="none"/>
  <rect x="100" y="100" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="150" width="50" height="50" stroke="black" fill="none"/>
</svg>
"""

# Candidate 7: Shifted vertical net
candidate7 = """
<svg viewBox="0 0 150 200" xmlns="http://www.w3.org/2000/svg">
  <!-- Candidate 7: Shifted vertical net -->
  <rect x="100" y="0" width="50" height="50" stroke="black" fill="none"/>
  <rect x="0" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="100" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="100" y="100" width="50" height="50" stroke="black" fill="none"/>
  <rect x="100" y="150" width="50" height="50" stroke="black" fill="none"/>
</svg>
"""

# Candidate 8: Left-column net
candidate8 = """
<svg viewBox="0 0 150 150" xmlns="http://www.w3.org/2000/svg">
  <!-- Candidate 8: Left-column net -->
  <rect x="0" y="0" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="0" width="50" height="50" stroke="black" fill="none"/>
  <rect x="100" y="0" width="50" height="50" stroke="black" fill="none"/>
  <rect x="0" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="0" y="100" width="50" height="50" stroke="black" fill="none"/>
</svg>
"""

# Candidate 9: Net with shifted coordinates
candidate9 = """
<svg viewBox="50 50 150 150" xmlns="http://www.w3.org/2000/svg">
  <!-- Candidate 9: Net with shifted coordinates -->
  <rect x="50" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="100" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="150" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="100" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="150" width="50" height="50" stroke="black" fill="none"/>
  <rect x="100" y="150" width="50" height="50" stroke="black" fill="none"/>
</svg>
"""

# Candidate 10: Central row with top and bottom attachments
candidate10 = """
<svg viewBox="0 0 150 150" xmlns="http://www.w3.org/2000/svg">
  <!-- Candidate 10: Central row with top and bottom attachments -->
  <rect x="50" y="0" width="50" height="50" stroke="black" fill="none"/>
  <rect x="100" y="0" width="50" height="50" stroke="black" fill="none"/>
  <rect x="0" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="100" y="50" width="50" height="50" stroke="black" fill="none"/>
  <rect x="50" y="100" width="50" height="50" stroke="black" fill="none"/>
</svg>
"""

# List of candidate unfoldings
candidate_unfoldings = [
    candidate0, candidate1, candidate2, candidate3, candidate4,
    candidate5, candidate6, candidate7, candidate8, candidate9, candidate10
]

# Display each candidate using the helper function with a fixed width.
for i, candidate in enumerate(candidate_unfoldings):
    print(f"Candidate {i}:")
    display_svg(candidate, width=200)


Candidate 0:


Candidate 1:


Candidate 2:


Candidate 3:


Candidate 4:


...

Candidate 6:


Candidate 7:


Candidate 8:


Candidate 9:


Candidate 10:


Main attributes for tweaking computation and results.
> **tolerance**:
- When converting SVG paths and shapes into polygons, Packaide “discretizes” (or approximates) the curves and lines by generating a set of points. The tolerance parameter controls how finely this discretization is done.
- Smaller Tolerance:
Generates more points per path, leading to a more accurate (and possibly more complex) polygon approximation.
Can yield tighter and more precise packings because the shape is represented more exactly.
However, it may increase computation time because there are more points to process.
- Larger Tolerance:
Results in a coarser approximation of the shape.
May run faster, but it could lead to a less precise representation, potentially affecting the quality of the packing (i.e., the parts might not interlock as tightly).

> **offset**:
- The offset parameter specifies the extra space (or “margin”) that is added around each shape when it is being packed. This is equivalent to “dilating” the shape by a fixed distance.
- Higher Offset:
Increases the gap between placed parts.
Ensures that there’s a safety margin so the parts don’t overlap—even when there might be slight inaccuracies.
However, this extra space may lead to a less efficient use of the material (more waste).
- Lower Offset:
Minimizes the gap between parts, potentially increasing material utilization.
But there’s a higher risk of overlap if the discretization (tolerance) isn’t precise enough or if slight errors occur in the packing process.

> **partial_solution**:
- When partial_solution is set to True, Packaide will return a solution that places as many parts as possible on the given sheet—even if it cannot place all of them. If set to False, Packaide will only return a result if it can place every part; otherwise, no solution is returned.
- True:
Provides a solution even when some parts cannot be placed.
Useful when you want to maximize usage and are willing to accept that some parts may remain unplaced.
- False:
Only acceptable if every part fits.
If not all parts can be placed, you get no solution at all.

> **rotations**:
- The rotations parameter tells Packaide how many orientations (rotations) of the shapes to consider when trying to pack them. A value of 1 means that only the original orientation is used.
- Higher Values:
Packaide will try additional rotations (e.g., for 4 rotations it might try 0°, 90°, 180°, and 270°).
This can lead to a better packing layout because a rotated shape might fit more snugly into the available space.
However, considering more rotations increases computation time.
- Value of 1:
Only the provided orientation is used.
Faster computation but might miss a more efficient layout that a rotated shape could provide.

> **persist**:
- When persist is set to True, Packaide caches (stores) computed results—especially the non-fit polygons (NFPs) or other intermediate geometry calculations—to speed up subsequent packing operations if the same shapes are encountered again.
- True:
Speeds up repeated packing operations, particularly useful in iterative or interactive settings.
Uses additional memory to store cached data.
- False:
No caching is done.
Each packing operation is computed from scratch, which might be slower if you’re working with many similar shapes repeatedly.

Below is a script that displays the packings of all original unfoldings along with each one's packaging according to packaide's internal logic.
> The reason we're displaying both is because we want to make sure that one of our original unfoldings is not already the most effiecient one before throwing it on packaide (which seems to be not super efficient at times).

In [ ]:
from IPython.display import HTML, display
from xml.dom import minidom
import packaide
import shapely.geometry

# --- Helper function to display an SVG string in a fixed-width container ---
def display_svg(svg_str, width=200):
    html_str = f'<div style="width: {width}px;">{svg_str}</div>'
    display(HTML(html_str))

# --- Function to add a bounding box overlay to an SVG string ---
def add_bounding_box_to_svg(svg_str, tolerance=1):
    # Compute the union polygon using Packaide's extraction
    _, polygons = packaide.extract_shapely_polygons(svg_str, tolerance)
    union_poly = None
    for boundary, _ in polygons:
        if union_poly is None:
            union_poly = boundary
        else:
            union_poly = union_poly.union(boundary)
    if union_poly is None:
        return svg_str  # nothing to overlay

    envelope = union_poly.envelope
    minx, miny, maxx, maxy = envelope.bounds
    width_rect = maxx - minx
    height_rect = maxy - miny

    # Parse the SVG string into a DOM document
    doc = minidom.parseString(svg_str)
    svg_elem = doc.getElementsByTagName("svg")[0]

    # Create a new <rect> element for the bounding box
    rect_elem = doc.createElement("rect")
    rect_elem.setAttribute("x", str(minx))
    rect_elem.setAttribute("y", str(miny))
    rect_elem.setAttribute("width", str(width_rect))
    rect_elem.setAttribute("height", str(height_rect))
    rect_elem.setAttribute("stroke", "red")
    rect_elem.setAttribute("fill", "none")
    rect_elem.setAttribute("stroke-dasharray", "4")

    # Append the bounding box rectangle to the SVG element
    svg_elem.appendChild(rect_elem)

    return doc.toxml()

# --- (Re)Use the geometry utility function to compute utilization ---
def compute_union_polygon(placed_svg, tolerance=1):
    _, polygons = packaide.extract_shapely_polygons(placed_svg, tolerance)
    union_poly = None
    for boundary, _ in polygons:
        if union_poly is None:
            union_poly = boundary
        else:
            union_poly = union_poly.union(boundary)
    return union_poly

def compute_utilization(placed_svg, tolerance=1):
    union_poly = compute_union_polygon(placed_svg, tolerance)
    if union_poly is None:
        return 0
    shape_area = union_poly.area
    bbox_area = union_poly.envelope.area
    if bbox_area == 0:
        return 0
    return shape_area / bbox_area

# --- Use your blank sheet (from previous cell) ---
sheet_width = 300
sheet_height = 300
sheet_svg = packaide.blank_sheet(sheet_width, sheet_height)

print("Displaying each candidate net in two versions:\n" +
      "1. The ORIGINAL candidate (with its own bounding box)\n" +
      "2. The PACKED version as processed by Packaide (with its bounding box)\n")

# Iterate over each candidate from candidate_unfoldings (assumed defined in Cell 1)
for i, candidate_svg in enumerate(candidate_unfoldings):
    print(f"\nCandidate {i}:")

    # --- ORIGINAL candidate version ---
    original_box_svg = add_bounding_box_to_svg(candidate_svg, tolerance=1)
    original_util = compute_utilization(candidate_svg, tolerance=1)
    print(f"  Original Utilization: {original_util:.2f}")
    display_svg(original_box_svg, width=200)

    # --- PACKED candidate version ---
    result, placed, fails = packaide.pack(
        [sheet_svg],
        candidate_svg,
        tolerance=1,
        offset=1,
        partial_solution=True,
        rotations=4,
        persist=False
    )
    if not result:
        print("  Packed version: Packing failed.")
        continue
    sheet_idx, packed_svg = result[0]
    packed_box_svg = add_bounding_box_to_svg(packed_svg, tolerance=1)
    packed_util = compute_utilization(packed_svg, tolerance=1)
    print(f"  Packed Utilization: {packed_util:.2f}")
    display_svg(packed_box_svg, width=200)


Displaying each candidate net in two versions:
1. The ORIGINAL candidate (with its own bounding box)
2. The PACKED version as processed by Packaide (with its bounding box)


Candidate 0:
  Original Utilization: 0.52


  Packed Utilization: 0.57



Candidate 1:
  Original Utilization: 0.52


  Packed Utilization: 0.57



Candidate 2:
  Original Utilization: 0.68


...

  Packed Utilization: 0.57



Candidate 9:
  Original Utilization: 0.69


  Packed Utilization: 0.57



Candidate 10:
  Original Utilization: 0.68


  Packed Utilization: 0.57


In [ ]:
import packaide
import shapely.geometry
from xml.dom import minidom
from IPython.display import HTML, display

# -------------------------------
# Utility Functions for Geometry Computation
# -------------------------------
def compute_union_polygon(placed_svg, tolerance=1):
    """
    Extract the polygons from the placed SVG (via Packaide) and return their union.
    """
    _, polygons = packaide.extract_shapely_polygons(placed_svg, tolerance)
    union_poly = None
    for boundary, _ in polygons:
        if union_poly is None:
            union_poly = boundary
        else:
            union_poly = union_poly.union(boundary)
    return union_poly

def compute_bounding_box_area(placed_svg, tolerance=1):
    """
    Compute the area of the minimal bounding rectangle (envelope) of the union polygon.
    """
    union_poly = compute_union_polygon(placed_svg, tolerance)
    if union_poly is None:
        return 0
    return union_poly.envelope.area

def compute_shape_area(placed_svg, tolerance=1):
    """
    Compute the total area of the union of all polygons in the placed SVG.
    """
    union_poly = compute_union_polygon(placed_svg, tolerance)
    return union_poly.area if union_poly is not None else 0

def compute_utilization(placed_svg, tolerance=1):
    """
    Compute the utilization ratio: (actual shape area) / (bounding box area).
    A value closer to 1 indicates less unused space.
    """
    shape_area = compute_shape_area(placed_svg, tolerance)
    bbox_area = compute_bounding_box_area(placed_svg, tolerance)
    if bbox_area == 0:
        return 0
    return shape_area / bbox_area

# -------------------------------
# Create a Blank Sheet (Material)
# -------------------------------
sheet_width = 300
sheet_height = 300
sheet_svg = packaide.blank_sheet(sheet_width, sheet_height)

# -------------------------------
# Evaluate Each Candidate
# -------------------------------
best_utilization = 0.0
best_candidate_index = None
best_nesting_result = None
tolerance = 1  # Discretization tolerance

# Assume candidate_unfoldings is already defined from Cell 1.
for i, candidate_svg in enumerate(candidate_unfoldings):
    result, placed, fails = packaide.pack(
        [sheet_svg],
        candidate_svg,
        tolerance=tolerance,
        offset=1,
        partial_solution=True,
        rotations=2,
        persist=False
    )

    if not result:
        print(f"Candidate {i} failed to pack.")
        continue

    sheet_idx, placed_svg = result[0]
    utilization = compute_utilization(placed_svg, tolerance)
    print(f"Candidate {i} utilization ratio: {utilization:.2f}")

    if utilization > best_utilization:
        best_utilization = utilization
        best_candidate_index = i
        best_nesting_result = placed_svg

# -------------------------------
# Display the Best Candidate
# -------------------------------
if best_candidate_index is not None:
    print(f"\nBest candidate: {best_candidate_index} with utilization ratio: {best_utilization:.2f}")
    # Use the helper function to display the best candidate with a fixed width.
    display_svg(best_nesting_result, width=200)
else:
    print("No candidate produced a valid nesting result.")


Candidate 0 utilization ratio: 0.57
Candidate 1 utilization ratio: 0.57
Candidate 2 utilization ratio: 0.57
Candidate 3 utilization ratio: 0.57
Candidate 4 utilization ratio: 0.57
Candidate 5 utilization ratio: 0.57
Candidate 6 utilization ratio: 0.57
Candidate 7 utilization ratio: 0.57
Candidate 8 utilization ratio: 0.57
Candidate 9 utilization ratio: 0.57
Candidate 10 utilization ratio: 0.57

Best candidate: 0 with utilization ratio: 0.57
